In [194]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [195]:
# download ethereum data from yfinance
eth = yf.download('AAPL', start='2015-08-07', end='2022-08-07')

# convert to pandas dataframe
eth = pd.DataFrame(eth)

eth.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-08-06,28.992500,29.125000,28.530001,28.782499,26.237539,211612000
2015-08-07,28.645000,29.062500,28.625000,28.879999,26.326424,154681600
2015-08-10,29.132500,29.997499,29.132500,29.930000,27.283579,219806400
2015-08-11,29.452499,29.545000,28.332500,28.372499,25.863798,388331200
2015-08-12,28.132500,28.855000,27.407499,28.809999,26.262609,404870000


In [196]:
def delete_consecutive_rows(df):
    # create a new dataframe to store the cleaned data
    cleaned_df = pd.DataFrame()
    
    # set the previous day to "none"
    prev_day = "none"
    
    # iterate through the rows of the dataframe
    for index, row in df.iterrows():
        # get the current day
        day = index.weekday()
        if day == 4:
            day = "friday"
        elif day == 0:
            day = "monday"
        else:
            continue
        
        # check if the current day is the same as the previous day
        if day != prev_day:
            # add the current row to the cleaned dataframe
            cleaned_df = cleaned_df.append(row)
            # update the previous day
            prev_day = day
    return cleaned_df


In [197]:
mask = (eth.index.weekday == 4) | (eth.index.weekday == 0)

eth = eth[mask]
eth = delete_consecutive_rows(eth)

/tmp/ipykernel_12273/1272178690.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_df = cleaned_df.append(row)
/tmp/ipykernel_12273/1272178690.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_df = cleaned_df.append(row)
/tmp/ipykernel_12273/1272178690.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_df = cleaned_df.append(row)
/tmp/ipykernel_12273/1272178690.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_df = cleaned_df.append(row)
/tmp/ipykernel_12273/1272178690.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conc

In [198]:
# drop every date that is not friday or monday
# Create a boolean mask that filters out the desired dates
mask_friday = (eth.index.weekday == 4)
mask_monday = (eth.index.weekday == 0)

# use the boolean mask to filter the dataframe and keep only friday and monday
eth_x = eth[mask_friday]
eth_y = eth[mask_monday]

print(eth_x.head())
print(eth_y.head())



               Open       High        Low      Close  Adj Close       Volume
2015-08-07  28.6450  29.062500  28.625000  28.879999  26.326424  154681600.0
2015-08-14  28.5800  29.077499  28.502501  28.990000  26.426689  171718000.0
2015-08-21  27.6075  27.975000  26.412500  26.440001  24.102167  513102000.0
2015-08-28  28.0425  28.327499  27.885000  28.322500  25.818211  212657600.0
2015-09-04  27.2425  27.612499  27.127501  27.317499  24.902079  199985200.0
                 Open       High      Low      Close  Adj Close       Volume
2015-08-10  29.132500  29.997499  29.1325  29.930000  27.283579  219806400.0
2015-08-17  29.010000  29.412500  28.8750  29.290001  26.700169  163538800.0
2015-08-24  23.717501  27.200001  23.0000  25.780001  23.500523  648825200.0
2015-08-31  28.007500  28.632500  28.0000  28.190001  25.697428  224917200.0
2015-09-14  29.145000  29.222500  28.7150  28.827499  26.278566  233453600.0


In [199]:
# change eth_x to percent change
eth_x = eth_x.pct_change()

# change inf to 0
eth_x = eth_x.replace([np.inf, -np.inf], 0)

# drop na
eth_x = eth_x.dropna()

# drop afjusted close
eth_x = eth_x.drop(['Adj Close'], axis=1)

In [200]:
eth_x

,Open,High,Low,Close,Volume
2015-08-14,-0.002269,0.000516,-0.004279,0.003809,0.110139
2015-08-21,-0.034027,-0.037916,-0.073327,-0.087961,1.988050
2015-08-28,0.015757,0.012601,0.055750,0.071199,-0.585545
2015-09-04,-0.028528,-0.025240,-0.027165,-0.035484,-0.059591
2015-09-18,0.029733,0.034857,0.030965,0.038254,0.485816
...,...,...,...,...,...
2022-07-01,0.045898,0.044785,0.045066,0.056020,-0.471815
2022-07-15,0.101000,0.085012,0.092437,0.080904,0.073303
2022-07-22,0.037455,0.035927,0.035155,0.026104,-0.125682
2022-07-29,0.037647,0.047031,0.039698,0.054643,0.526604


In [201]:
# save the eth_x index
eth_x_index = eth_x.index
print(eth_x_index)
print(len(eth_x_index))

# save the eth_x columns
eth_x_columns = eth_x.columns

# scale the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
eth_x = scaler.fit_transform(eth_x)

# put eth_x to pandas dataframe
eth_x = pd.DataFrame(eth_x, columns=eth_x_columns, index=eth_x_index)

eth_x

DatetimeIndex(['2015-08-14', '2015-08-21', '2015-08-28', '2015-09-04',
               '2015-09-18', '2015-09-25', '2015-10-02', '2015-10-09',
               '2015-10-16', '2015-10-23',
               ...
               '2022-05-13', '2022-05-20', '2022-05-27', '2022-06-10',
               '2022-06-17', '2022-07-01', '2022-07-15', '2022-07-22',
               '2022-07-29', '2022-08-05'],
              dtype='datetime64[ns]', length=316, freq=None)
316


,Open,High,Low,Close,Volume
2015-08-14,0.540399,0.466658,0.553230,0.516544,0.171446
2015-08-21,0.456897,0.344192,0.363785,0.251892,0.540708
2015-08-28,0.587795,0.505166,0.717933,0.710887,0.034651
2015-09-04,0.471356,0.384583,0.490439,0.403228,0.138072
2015-09-18,0.624543,0.576090,0.649930,0.615878,0.245317
...,...,...,...,...,...
2022-07-01,0.667047,0.607724,0.688619,0.667113,0.057014
2022-07-15,0.811927,0.735909,0.818591,0.738876,0.164203
2022-07-22,0.644847,0.579499,0.661427,0.580839,0.125076
2022-07-29,0.645353,0.614881,0.673890,0.663143,0.253337


In [202]:

# drop from y all other values than open
eth_y = eth_y.drop(['High', 'Low', 'Close', 'Volume', "Adj Close"], axis=1)

# remove datetimeindex from eth_y
eth_y = eth_y.reset_index()

# rename eth_y column to target_open
eth_y = eth_y.rename(columns={'Open': 'target_open'})

# set eth_y's index the index from eth_x
eth_y = eth_y.set_index(eth_x.index)

# join eth_x and eth_y
eth = eth_x.join(eth_y)

eth['target_open'] = eth['target_open'].shift(1)

# drop date from eth
eth = eth.drop(['index'], axis=1)

# drop na
eth = eth.dropna()

eth

,Open,High,Low,Close,Volume,target_open
2015-08-21,0.456897,0.344192,0.363785,0.251892,0.540708,29.132500
2015-08-28,0.587795,0.505166,0.717933,0.710887,0.034651,29.010000
2015-09-04,0.471356,0.384583,0.490439,0.403228,0.138072,23.717501
2015-09-18,0.624543,0.576090,0.649930,0.615878,0.245317,28.007500
2015-09-25,0.645484,0.531645,0.617702,0.537588,0.101790,29.145000
...,...,...,...,...,...,...
2022-07-01,0.667047,0.607724,0.688619,0.667113,0.057014,132.869995
2022-07-15,0.811927,0.735909,0.818591,0.738876,0.164203,142.699997
2022-07-22,0.644847,0.579499,0.661427,0.580839,0.125076,145.669998
2022-07-29,0.645353,0.614881,0.673890,0.663143,0.253337,150.740005


In [203]:
# split into x and y, y is target_open
x = eth.drop(['target_open'], axis=1)
y = eth['target_open']

# split into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

In [204]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# create DMatrix
dtrain = xgb.DMatrix(x_train, label=y_train)

# create parameter grid
param_grid = {
    'max_depth': [3, 5, 7],
    'eta': [0.01, 0.1, 0.3, 0.5, 0.7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'lambda': [1, 2, 4],
    'alpha': [0, 1, 2],
    'objective': ['reg:squarederror'],
    'eval_metric': ['rmse']
}

# create XGBoost model
xgb_model = xgb.XGBRegressor()

# create GridSearchCV
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, 
                           scoring='neg_mean_squared_error', 
                           return_train_score=True)

# fit to data
grid_search.fit(x_train, y_train)

# view best parameters
print(grid_search.best_params_)

# create DMatrix for test data
dtest = xgb.DMatrix(x_test)

# predict using best parameters
y_pred = grid_search.predict(x_test)

# plot, y_test index to match y_pred
plt.plot(y_test.index, y_pred, label='Predicted')
plt.plot(y_test.index, y_test, label='Actual')
plt.legend()
plt.show()

# calculate rmse
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))


KeyboardInterrupt: 